In [1]:
import requests 
import time
from tqdm import tqdm
import json
import pandas as pd
import os

In [59]:
collection = 'GSPACEAPE-08bc2b'
chunk_size = 50
sleep_time = 0.4

In [3]:
data = pd.read_csv(f'{collection}_nfts.csv')

In [4]:
if f'{collection}_transactions.json' in os.listdir():
    with open(f'{collection}_transactions.json', 'r') as f:
        transactions_data = json.load(f)
else:
    transactions_data = {}

In [66]:
for _, nft in tqdm(data.iterrows(), total=data.shape[0]):
    identifier = nft['identifier']
    if identifier in transactions_data and transactions_data[identifier]['count'] == len(transactions_data[identifier]['transactions']):
        continue
    transactions_data[identifier] = {
        'count': -1,
        'transactions': []
    }
    time.sleep(sleep_time)
    url = f'https://api.elrond.com/nfts/{identifier}/transactions/count'
    count = requests.get(url).json()
    transactions_data[identifier]['count'] = count
    for index in range(0, count, chunk_size):
        time.sleep(sleep_time)
        url = f'https://api.elrond.com/nfts/{identifier}/transactions?from={index}&size={chunk_size}'
        transactions_data[identifier]['transactions'] += requests.get(url).json()

100%|██████████| 14166/14166 [00:00<00:00, 23649.70it/s]


In [67]:
with open(f'{collection}_transactions.json', 'w') as f:
    json.dump(transactions_data, f, indent=4)

In [68]:
c = 0 
for identifier in transactions_data:
    c += transactions_data[identifier]['count']
c

110566